In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, top_k_accuracy_score


2024-09-24 23:51:28.107079: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 23:51:28.107115: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 23:51:28.107138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 23:51:28.112128: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
'''Local Imports'''
from cap.custom_validate_callback import CustomCallback
from cap.image_data_generator import DirectoryDataGenerator
from cap.loupe_keras import NetRVLAD
from cap.roi_pooling_conv import RoiPoolingConv
from cap.self_attention import SelfAttention
from cap.seq_attention import SeqSelfAttention
from cap.spectral_normalization import ConvSN2D
from cap.se import squeeze_excite_block

In [3]:
batch_size = 12
checkpoint_freq = 5
dataset_dir = "./datasets/food-101-3splits"
epochs = 10 # 150
image_size = (224,224)
lstm_units = 128
model_name = "CAP_Xception"
nb_classes = 101
optimizer = SGD(learning_rate=0.0001, momentum=0.99, nesterov=True)
train_dir = "{}/train".format(dataset_dir)
val_dir = "{}/val".format(dataset_dir)
validation_freq = 15 # 5

2024-09-24 23:51:30.177128: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22108 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:c1:00.0, compute capability: 8.9


In [4]:
'''Model Methods'''
#get regions of interest of an image (return all possible bounding boxes when splitting the image into a grid)
def getROIS(resolution=33,gridSize=3, minSize=1):
	
	coordsList = []
	step = resolution / gridSize # width/height of one grid square
	
	#go through all combinations of coordinates
	for column1 in range(0, gridSize + 1):
		for column2 in range(0, gridSize + 1):
			for row1 in range(0, gridSize + 1):
				for row2 in range(0, gridSize + 1):
					
					#get coordinates using grid layout
					x0 = int(column1 * step)
					x1 = int(column2 * step)
					y0 = int(row1 * step)
					y1 = int(row2 * step)
					
					if x1 > x0 and y1 > y0 and ((x1 - x0) >= (step * minSize) or (y1 - y0) >= (step * minSize)): #ensure ROI is valid size
						
						if not (x0==y0==0 and x1==y1==resolution): #ignore full image
							
							#calculate height and width of bounding box
							w = x1 - x0
							h = y1 - y0
							
							coordsList.append([x0, y0, w, h]) #add bounding box to list

	coordsArray = np.array(coordsList)	 #format coordinates as numpy array						

	return coordsArray

def crop(dimension, start, end): #https://github.com/keras-team/keras/issues/890
    #Use this layer for a model that has individual roi bounding box
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return layers.Lambda(func)

def squeezefunc(x):
    return K.squeeze(x, axis=1)

def squeeze_output_shape(input_shape):
    # transforming shape from (None, 1, 7, 7, 2048) to (1, 7, 7, 2048) - (axis=1) 
    return (input_shape[0], input_shape[2], input_shape[3], input_shape[4])

'''This is to convert stacked tensor to sequence for LSTM'''
def stackfunc(x):
    return K.stack(x, axis=1) 

def stack_output_shape(input_shapes):
    # Input_shapes is a list of tensor shapes
    # We stack them along a new axis, so we add an extra axis for stacking
    return (None, len(input_shapes), input_shapes[0][1])

In [5]:
'''Model Constants'''
ROIS_resolution = 42
ROIS_grid_size = 3
min_grid_size = 2
pool_size = 7
loss_type = 'categorical_crossentropy'
metrics = ['accuracy']

In [6]:
'''Model Construction'''
base_model = Xception(weights='imagenet', input_tensor=layers.Input(shape=(image_size[0],image_size[1],3)), include_top=False)

#last convolution layer
base_out = base_model.output

dims = base_out.shape[1:] # dims = base_out.shape.as_list()[1:]
feat_dim = dims[2]*pool_size*pool_size
base_channels = dims[2]

x = base_out
x = squeeze_excite_block(x) #Added new

#self-attention
x_f = ConvSN2D(base_channels//8, kernel_size=1, strides=1, padding='same')(x)# [bs, h, w, c']
x_g = ConvSN2D(base_channels//8, kernel_size=1, strides=1, padding='same')(x) # [bs, h, w, c']
x_h = ConvSN2D(base_channels, kernel_size=1, strides=1, padding='same')(x)
x_final = SelfAttention(filters=base_channels)([x, x_f, x_g, x_h])

#x_final = base_out

# full_img = layers.Lambda(lambda x: K.tf.image.resize_images(x,size=(ROIS_resolution, ROIS_resolution)), name='Lambda_img_1')(x_final) #Use bilinear upsampling (default tensorflow image resize) to a reasonable size
full_img = layers.Lambda(lambda x: tf.image.resize(x, size=(ROIS_resolution, ROIS_resolution)), 
                         output_shape=(ROIS_resolution, ROIS_resolution, base_channels), 
                         name='Lambda_img_1')(x_final)


"""Do the ROIs information and separate them out"""
rois_mat =  getROIS(resolution=ROIS_resolution,gridSize=ROIS_grid_size, minSize=min_grid_size)
num_rois = rois_mat.shape[0]

roi_pool = RoiPoolingConv(pool_size=pool_size, num_rois=num_rois, rois_mat=rois_mat)(full_img)

jcvs = []
for j in range(num_rois):
    roi_crop = crop(1, j, j+1)(roi_pool)
    lname = 'roi_lambda_'+str(j)
    # x = layers.Lambda(squeezefunc, name=lname)(roi_crop)
    x = layers.Lambda(squeezefunc, output_shape=squeeze_output_shape, name=lname)(roi_crop)
    x = layers.Reshape((feat_dim,))(x)
    jcvs.append(x)
x = layers.Reshape((feat_dim,))(x_final)
jcvs.append(x)

jcvs = layers.Lambda(stackfunc, output_shape=stack_output_shape, name='lambda_stack')(jcvs)
x = SeqSelfAttention(units=32, attention_activation='sigmoid', name='Attention')(jcvs) 

x = layers.TimeDistributed(layers.Reshape((pool_size,pool_size, base_channels)))(x)

x = layers.TimeDistributed(layers.GlobalAveragePooling2D(name='GAP_time'))(x)

lstm = layers.LSTM(lstm_units, return_sequences=True)(x) #experiment with different units 64, 256 etc

y = NetRVLAD(feature_size=128, max_samples=num_rois+1, cluster_size=32, output_dim=nb_classes)(lstm)
y = layers.BatchNormalization(name='batch_norm_last')(y)
y = layers.Activation('softmax', name='final_softmax')(y)

model = Model(base_model.input, y)
model.compile(loss="categorical_crossentropy", metrics=['accuracy'], optimizer=optimizer)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 111, 111, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 111, 111, 32)         0         ['block1_conv1_bn[0][0]'] 

In [7]:
''' Training information '''
try:
    os.mkdir("./Metrics")
    os.mkdir("./TrainedModels")
except:
    pass

#Every 50 epochs reduce the learning
def epoch_decay(epoch):
    my_lr = K.eval(model.optimizer.lr)
    if epoch % 50 == 0 and not epoch == 0:
       my_lr = my_lr / 10
    print("EPOCH: ", epoch, "Current LR: ", my_lr)
    return my_lr

In [8]:
basic_schedule = LearningRateScheduler(epoch_decay)

# metrics_dir = './Metrics/{}'.format(model_name)
metrics_dir = './Metrics/'
output_model_dir = './TrainedModels/{}'.format(model_name)

csv_logger = CSVLogger(metrics_dir + 'training_epochs.csv')
checkpointer = ModelCheckpoint(filepath = output_model_dir + '.{epoch:02d}.keras', verbose=1, save_weights_only=False, save_freq='epoch')

In [9]:
# !utils/split_food101.py

nb_train_samples = sum([len(files) for r, d, files in os.walk(train_dir)])
nb_val_samples = sum([len(files) for r, d, files in os.walk(val_dir)])

train_dg = DirectoryDataGenerator(base_directories=[train_dir], augmentor=True, target_sizes=image_size, preprocessors=preprocess_input, batch_size=batch_size, shuffle=True)
val_dg = DirectoryDataGenerator(base_directories=[val_dir], augmentor=False, target_sizes=image_size, preprocessors=preprocess_input, batch_size=batch_size, shuffle=False)

print("train images: ", nb_train_samples)
print("val images: ", nb_val_samples)

Using label 0 for class apple_pie
Using label 1 for class baby_back_ribs
Using label 2 for class baklava
Using label 3 for class beef_carpaccio
Using label 4 for class beef_tartare
Using label 5 for class beet_salad
Using label 6 for class beignets
Using label 7 for class bibimbap
Using label 8 for class bread_pudding
Using label 9 for class breakfast_burrito
Using label 10 for class bruschetta
Using label 11 for class caesar_salad
Using label 12 for class cannoli
Using label 13 for class caprese_salad
Using label 14 for class carrot_cake
Using label 15 for class ceviche
Using label 16 for class cheese_plate
Using label 17 for class cheesecake
Using label 18 for class chicken_curry
Using label 19 for class chicken_quesadilla
Using label 20 for class chicken_wings
Using label 21 for class chocolate_cake
Using label 22 for class chocolate_mousse
Using label 23 for class churros
Using label 24 for class clam_chowder
Using label 25 for class club_sandwich
Using label 26 for class crab_cake

In [10]:
!rm Metrics/*
!rm TrainedModels/*

rm: cannot remove 'Metrics/*': No such file or directory
rm: cannot remove 'TrainedModels/*': No such file or directory


In [11]:
!rm lastmodel.h5

model.fit(
    train_dg, 
    steps_per_epoch=nb_train_samples // batch_size,  
    epochs=epochs,
    validation_freq=1,
    callbacks=[checkpointer, csv_logger]
    # callbacks=[checkpointer, csv_logger, CustomCallback(val_dg, validation_freq, metrics_dir)]
)
model.save('lastmodel.h5')

rm: cannot remove 'lastmodel.h5': No such file or directory
Epoch 1/10


2024-09-24 23:51:43.658901: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-09-24 23:51:45.536509: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb550273240 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-24 23:51:45.536528: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-09-24 23:51:45.588918: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6312/6312 [==============================] - ETA: 0s - loss: 4.0377 - accuracy: 0.1274
Epoch 1: saving model to ./TrainedModels/CAP_Xception.01.keras
6312/6312 [==============================] - 497s 76ms/step - loss: 4.0377 - accuracy: 0.1274
Epoch 2/10
6312/6312 [==============================] - ETA: 0s - loss: 2.8766 - accuracy: 0.3816
Epoch 2: saving model to ./TrainedModels/CAP_Xception.02.keras
6312/6312 [==============================] - 481s 76ms/step - loss: 2.8766 - accuracy: 0.3816
Epoch 3/10
6312/6312 [==============================] - ETA: 0s - loss: 2.1573 - accuracy: 0.5349
Epoch 3: saving model to ./TrainedModels/CAP_Xception.03.keras
6312/6312 [==============================] - 482s 76ms/step - loss: 2.1573 - accuracy: 0.5349
Epoch 4/10
6312/6312 [==============================] - ETA: 0s - loss: 1.6845 - accuracy: 0.6312
Epoch 4: saving model to ./TrainedModels/CAP_Xception.04.keras
6312/6312 [==============================] - 482s 76ms/step - loss: 1.6845 - accuracy

/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
custom_obj = {
    'ConvSN2D': ConvSN2D, 
    'SelfAttention': SelfAttention,
    'SeqSelfAttention': SeqSelfAttention,
    'RoiPoolingConv': RoiPoolingConv,
    'NetRVLAD': NetRVLAD
}
# last_model = load_model('lastmodel.h5', compile=False, safe_mode=False, custom_objects=custom_obj)

In [13]:
test_dir = "{}/test".format(dataset_dir)
class_labels = sorted(os.listdir(test_dir))

In [14]:
test_dg = DirectoryDataGenerator(base_directories=[test_dir], augmentor=False, target_sizes=image_size, preprocessors=preprocess_input, batch_size=batch_size, shuffle=False)

Using label 0 for class apple_pie
Using label 1 for class baby_back_ribs
Using label 2 for class baklava
Using label 3 for class beef_carpaccio
Using label 4 for class beef_tartare
Using label 5 for class beet_salad
Using label 6 for class beignets
Using label 7 for class bibimbap
Using label 8 for class bread_pudding
Using label 9 for class breakfast_burrito
Using label 10 for class bruschetta
Using label 11 for class caesar_salad
Using label 12 for class cannoli
Using label 13 for class caprese_salad
Using label 14 for class carrot_cake
Using label 15 for class ceviche
Using label 16 for class cheese_plate
Using label 17 for class cheesecake
Using label 18 for class chicken_curry
Using label 19 for class chicken_quesadilla
Using label 20 for class chicken_wings
Using label 21 for class chocolate_cake
Using label 22 for class chocolate_mousse
Using label 23 for class churros
Using label 24 for class clam_chowder
Using label 25 for class club_sandwich
Using label 26 for class crab_cake

In [15]:
def evaluate_model(model, test_data_generator, class_labels, output_path):
    y_true = []
    y_pred = []

    # collect labels
    for i in range(len(test_data_generator)):
        _, labels = test_data_generator[i]  
        y_true.extend(np.argmax(labels, axis=1))  # one-hot labels --> class indices

    # predict
    y_pred_prob = model.predict(test_data_generator, steps=len(test_data_generator), verbose=1)
    y_pred = np.argmax(y_pred_prob, axis=1)  # probabilities --> predicted class indices

    # generate metrics
    report = classification_report(y_true, y_pred, target_names=class_labels, output_dict=True)
    accuracy = accuracy_score(y_true, y_pred)
    top_3_accuracy = top_k_accuracy_score(y_true, y_pred_prob, k=3)
    top_5_accuracy = top_k_accuracy_score(y_true, y_pred_prob, k=5)
    conf_matrix = confusion_matrix(y_true, y_pred)

    # save metrics
    report_df = pd.DataFrame(report).transpose()
    report_df['accuracy'] = accuracy
    report_df['top_3_accuracy'] = top_3_accuracy
    report_df['top_5_accuracy'] = top_5_accuracy
    report_df.to_csv(os.path.join(output_path, 'metrics.csv'), index=True)

    print("Confusion Matrix:\n", conf_matrix)
    print(f"Accuracy: {accuracy}")
    print(f"Top-3 Accuracy: {top_3_accuracy}")
    print(f"Top-5 Accuracy: {top_5_accuracy}")

    return y_pred

In [16]:
preds = evaluate_model(model, test_dg, class_labels, metrics_dir)

2104/2104 [==============================] - 56s 26ms/step
Confusion Matrix:
 [[250   0   0 ...   0   0   0]
 [  2 248   0 ...   0   0   0]
 [  0   2 248 ...   0   0   0]
 ...
 [  0   0   0 ... 246   0   0]
 [  0   0   0 ...   6 238   4]
 [  0   0   0 ...   0   0 248]]
Accuracy: 0.9887515842839036
Top-3 Accuracy: 0.9994455006337135
Top-5 Accuracy: 0.9998019645120405


In [17]:
unique_values, counts = np.unique(preds, return_counts=True)

table = np.column_stack((unique_values, counts))

print("Value | Count")
print(table)

Value | Count
[[  0 252]
 [  1 250]
 [  2 254]
 [  3 246]
 [  4 243]
 [  5 252]
 [  6 252]
 [  7 252]
 [  8 249]
 [  9 246]
 [ 10 249]
 [ 11 252]
 [ 12 252]
 [ 13 252]
 [ 14 252]
 [ 15 241]
 [ 16 252]
 [ 17 252]
 [ 18 252]
 [ 19 250]
 [ 20 252]
 [ 21 252]
 [ 22 242]
 [ 23 252]
 [ 24 252]
 [ 25 252]
 [ 26 246]
 [ 27 246]
 [ 28 252]
 [ 29 252]
 [ 30 252]
 [ 31 252]
 [ 32 240]
 [ 33 252]
 [ 34 252]
 [ 35 252]
 [ 36 252]
 [ 37 261]
 [ 38 254]
 [ 39 240]
 [ 40 252]
 [ 41 254]
 [ 42 252]
 [ 43 252]
 [ 44 241]
 [ 45 253]
 [ 46 250]
 [ 47 252]
 [ 48 252]
 [ 49 252]
 [ 50 245]
 [ 51 248]
 [ 52 251]
 [ 53 252]
 [ 54 252]
 [ 55 252]
 [ 56 250]
 [ 57 242]
 [ 58 252]
 [ 59 250]
 [ 60 252]
 [ 61 251]
 [ 62 242]
 [ 63 251]
 [ 64 252]
 [ 65 252]
 [ 66 252]
 [ 67 247]
 [ 68 245]
 [ 69 252]
 [ 70 252]
 [ 71 252]
 [ 72 252]
 [ 73 250]
 [ 74 242]
 [ 75 254]
 [ 76 250]
 [ 77 251]
 [ 78 252]
 [ 79 250]
 [ 80 242]
 [ 81 254]
 [ 82 250]
 [ 83 252]
 [ 84 252]
 [ 85 250]
 [ 86 254]
 [ 87 240]
 [ 88 252]
 [ 89 2